# Объединение файлов и фильтрация данных

## Постановка задачи

#### Имеются файлы (tm_sales_1, tm_sales_2, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

#### Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

#### Особенности данных:

- **сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)**
- **если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается**

#### Сохраните результат в датафрэйм с разделителем ";", содержащий корректные подключения.

*Пример содержания итогового файла (колонка difference это разница между START_DTTM и END_DTTM):*
```
ACT_DTTM;FILIAL_ID;difference
20-03-2020 14:59;1;255 days 09:01:00
```

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
tree = os.walk(r'D:\data_analysis\projects\subsid_to_combine')

# Создаем пустой датафрейм
full_df = pd.DataFrame()

# Считываем файлы и объединяем все датафреймы в итоговый
for path, dirs, files in tree:                                                       
    for file in files:                                                      
        file_path = '\\'.join([path, file]) 
        
        if file.startswith('tm'):
            new_df = pd.read_csv(file_path, sep=';', parse_dates=['ACT_DTTM'])
            new_df.dropna(inplace=True)
            new_df['SUBS_ID'] = 'id' + new_df.SUBS_ID.str.extract('(\d+)')
            full_df = pd.concat([full_df, new_df], ignore_index=True)
            
        else:                                                                  
            logs_df = pd.read_csv(file_path, sep=';')
                                                                                                
    full_df = pd.merge(full_df, logs_df, on='SUBS_ID')
    full_df = full_df.assign(START_DTTM = pd.to_datetime(full_df.START_DTTM, dayfirst=True),
                             END_DTTM = pd.to_datetime(full_df.END_DTTM, dayfirst=True))
    
    full_df['difference'] = full_df.END_DTTM - full_df.START_DTTM
    full_df = full_df.loc[full_df.difference > '5m']
        
full_df

,SUBS_ID,FILIAL_ID,PROD_ID_x,ACT_DTTM,PROD_ID_y,START_DTTM,END_DTTM,difference
0,id4651830,1,1954,2020-03-20 14:59:00,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id5416547,3,1743,2020-03-17 10:17:00,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,2020-03-16 16:28:00,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,2020-03-15 14:21:00,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [3]:
# Создадим итоговый датасет с требуемыми данными и нужными колонками
path = r'D:\data_analysis\projects\subsid_to_combine\final_data.csv'
necessary_df = full_df[['ACT_DTTM', 'FILIAL_ID', 'difference']].to_csv(path, sep=';', index=False)
pd.read_csv(path, sep=';')

,ACT_DTTM,FILIAL_ID,difference
0,2020-03-20 14:59:00,1,255 days 09:01:00
1,2020-03-17 10:17:00,3,8 days 00:43:00
2,2020-03-16 16:28:00,2,259 days 07:32:00
3,2020-03-15 14:21:00,6,0 days 09:21:00
